## Optimization Algorithms
As deep learning is an empirical process that requires many iterations and much experimentation it is important to be able to train networks quickly as this expedites the development process.

Vectorization is a MUST DO optimization practice that allows us to write code without explicit for loops as we can perform matrix and vector math instead.

### Mini-batch gradient descent
Instead of running the entire training set through the network you instead allow graidient descent to happen after the model processes a portion (or batch) of the training set. This is particularally useful for when you have a large data set (say over a million training examples). 

The smaller subset of the training data are called **mini-batches**

The notation for distinguishing between the different mini batches we use:

x{1}, x{2}, x{l}

y{1}, y{2}, y{l}

Using mini-batch gradient descent trains the network much quicker and comes with few downsides. This approach is almost always used when training networks on large datasets.

When plotting the cost function of mini-batch gradient descent the cost will contain much more noise than when using Batch gradient descent as each mini-batch is completely new data.

#### Choosing the Mini-Batch size
When the mini-batch size is equal to m you are effectively implementing a Batch gradient descent

When the mini-batch size is equal to 1 you are implementing **stochastic gradient descent** 
- this method never converges, but moves around with noise constantly
- this approach does not really result in an increase in network training speed and provides less oppertunity to vectorize math (you need for loops)

SGD leads to many oscillations to reach convergence, but each step is a lot faster to compute for SGD than it is for GD, as it uses only one training example (vs. the whole batch for GD).
- Over the number of iterations
- Over the  𝑚 training examples
- Over the layers (to update all parameters, from  (𝑊[1],𝑏[1]) to  (𝑊[𝐿],𝑏[𝐿]))

- When implementing the mini-batching you want to randomly shuffle your X and Y data so that X(1) is moved to the same new position as Y(1)
- Next, partician the shuffled (X, Y) values into min-batches of size mini_batch_size
- It is likely that the number of taining examples does not evenly divide by mini_batch_size. But this is not a problem as the last batch will simply be smaller than the prior batches

**Note also that implementing SGD requires 3 for-loops in total**

If you have a small training set (< 2000): use Batch gradient descent
Typical mini-batch sizes are a power of 2 of 64, 128, 256, 512, (1024 sometimes, but it is more rare)
**MAKE SURE THE MINI BATCH FITS WITHIN CPU/GPU MEMORY OR ELSE THE PERFORMANCE WILL TANK**

## Exponentially Weighted (moving) Averages
You calculate an average for each point in the dataset using some weight.

- theta = data points
- Vt = weighted average of datapoint t
- beta = the weight value

Vt = (beta) * V(t-1) + (1 - beta) * theta(t)

If beta = 0.9:

V1 = 0.9 * V0 + 0.1(theta1)

V2 = 0.9 * V1 + 0.1(theta2)

You are basically limiting the amount that the current value contributes to the average by comparing it to the values which came before to create a localized average.

This takes very little memory, and with quite efficent and easy to implement.

This is not the most accurate way to calculate the average, but it is computationally cheep and generally provides pretty good results.

### Choosing values for Beta
0.9 is the most common value used for beta

**IMPORTANT** Vt provides an approximate average over 1/(1-beta) days
- With a beta value of 0.9 this equates to 1/(1-0.9) = 1/0.1 = 10
- With a beta value of 0.95 this equates to 1/(1-0.95) = 1/0.05 = 20
- With a beta value of 0.75 this equates to 1/(1-0.75) = 1/0.25 = 4

The problem with using a high beta value is that it will effectively right-shift the average away from the data.

### Bias correction for Weighted Averages
A problem with weighted averages is that until you feed 1 / (1 - beta) data points into to algorithm you will be calculating a lower average than what is accurate (as V0 is initialized as 0)

**PERSONAL NOTE** I feel like this can be avoided by initializing V0 as the value of theta1.... I wonder if that is correct

To combat the bia issue for the weighted average you can apply this equasion to Vt:

Vt = Vt / (1 - beta^t)

## Gradient descent with momentum
Compute an exponentially weighted average of your gradients and then apply that instead of the unweighted gradients

On iteration t: compute dw and db on current mini-batch

- Vdw = (beta * Vdw) + ((1 - beta) * dw)
- Vdb = (beta * Vdb) + ((1 - beta) * db)
- W = W - learning_rate * Vdw
- b = b - learning_rate * Vdb

One way to conceptualize how the different parts of this function works is if you think of the cost function as tring to find the lowest points in a bowl and the current cost function is a ball. When using this analogy, dW and db can be thought of as the acceleration of the ball while Vdb functions as the velocity of the ball while beta serves as the friction that prevents the ball from moving too fast

### Implementation
Compute dW, db on the current mini-batch

VdW = beta * Vdw + ((1 - beta) * dW)
Vdb = beta * Vdb + ((1 - beta) * db)
W = W - (learning_rate * Vdw)
b = b - (learning_rate * Vdb)

Hyperparameters: learning_rate, beta

**PLEASE NOTE** That usually when gradient decent with momemtum is used bia correction does not occur. This is especially true with lower values of beta such as 0.9 as it only takes around ten iterations before there is no bias

## RMSprop (Root Means Square)
Has a similar effect to Momentum by reducing the oscilations 
On iteration t: computer dw, db on current mini-batch

epsilon = 10^-8

Sdw = (beta * Sdw) + ((1 - beta) * dw^2) (element wide square)
Sdb = (beta * Sdb) + ((1 - beta) * db^2)

W = W - (alpha * (dw/np.sqrt(Sdw + epsilon)))

**IMPORTANT** One huge advantage of this approach is that it allows you to use a higher learning rate than otherwise would be possible.
**HISTORY** RMSprop was not introduced to the DeepLearning community through an academic paper, but instead through a coursera course =)

## Combining RMSprop and Momentum (Adam optimization algorithm)

Adam = adaptive moment estimation

Vdw = 0, Sdw = 0, Vdb = 0, Sdb = 0

epsilon = 10^-8

On iteration t:
- compute dw, db using current mini-batch
- Vdw = beta1 * Vdw + ((1 - beta1) * dw)    <--- momentum implementation
- Vdb = beta1 * Vdb + ((1 - beta1) * db)    <--- momentum implementation
- Sdw = beta2 * Sdw + ((1 - beta2) * dw^2)  <--- RMSprop implementation
- Sdb = beta2 * Sdb + ((1 - beta2) * db^2)  <--- RMSprop implementation
- Vdw(corrected) = Vdw / (1 - beta2^t)
- Vdb(corrected) = Vdb / (1 - beta2^t)
- Sdw(corrected) = Sdw / (1 - beta2^t)
- Sdb(corrected) = Sdb / (1 - beta2^t)
- W = W - (learning_rate * (Vdw(corrected) / np.sqrt(Sdw(corrected) + epsilon)))
- b = b - (learning_rate * (Vdb(corrected) / np.sqrt(Sdb(corrected) _ epsilon)))

### Hyper-parameter choices
learning_rate = needs to be tuned
beta1 = 0.9 <--- to calculate dw
beta2 = 0.999 <--- to calculate dw^2
epsilon = 10^-8 <---- this specific value does not matter too much

## Learning Rate Decay
When you use the same learning rate for all batches and iterations through the network you might run into an issue where the cost function will not converge on the optima, when the model gets close to converging, it is unable to because the learning rate is too high to fully optimize the cost function.
- 1 epoch is one pass through the data (processing all mini-batches in the training set)
- learning_rate0 is the unmodified hyperparamter learning_rate
- learning_rate = 1 / (1 + decayRate * epochNumber) * learning_rate0

An alternate implementaiton of learning rate decay called exponential learning rate decay is implemented as follows:
- learning_rate = 0.95^epochnum * learning_rate

Another alternate approach is:
- learning_rate = k / np.sqrt(epochnum) * learning_rate

Another approach is:
- t is the minibatch number
- learning_rate = k / np.sqrt(t) * learning_rate0

The discrete staircase approach will half the learning rate after a set number of mini-batches are processed

Another method is manual and is used when a network takes many hours or days to train. For this approach you keep an eye on the output of the model as it trains and manually adjust the training rate as the network trains.

## Problems with local optima
This is mostly a problem with lower dimentional spaces, it is very unlickely in high dimensions that all dimensions will converge into a local optima instead of a global optima. Pleateaus can drastically slow down the model training however.
